# Elastic Collision Simulation

## Theoretical Background

An **elastic collision** is a collision in which both momentum and kinetic energy are conserved. This is an idealized scenario where no energy is lost to deformation, heat, or sound.

### Conservation Laws

#### Conservation of Momentum

For a system of two particles with masses $m_1$ and $m_2$, and velocities $\vec{v}_1$ and $\vec{v}_2$ before collision, and $\vec{v}_1'$ and $\vec{v}_2'$ after collision:

$$m_1 \vec{v}_1 + m_2 \vec{v}_2 = m_1 \vec{v}_1' + m_2 \vec{v}_2'$$

#### Conservation of Kinetic Energy

$$\frac{1}{2}m_1 v_1^2 + \frac{1}{2}m_2 v_2^2 = \frac{1}{2}m_1 v_1'^2 + \frac{1}{2}m_2 v_2'^2$$

### One-Dimensional Elastic Collision

For a head-on collision in one dimension, the final velocities can be derived analytically:

$$v_1' = \frac{(m_1 - m_2)v_1 + 2m_2 v_2}{m_1 + m_2}$$

$$v_2' = \frac{(m_2 - m_1)v_2 + 2m_1 v_1}{m_1 + m_2}$$

### Two-Dimensional Elastic Collision

For spherical particles colliding in 2D, we decompose velocities along the collision normal $\hat{n}$ (connecting centers) and tangent $\hat{t}$:

$$\hat{n} = \frac{\vec{r}_2 - \vec{r}_1}{|\vec{r}_2 - \vec{r}_1|}$$

The velocity components along the normal direction are exchanged according to the 1D formulas, while tangential components remain unchanged:

$$v_{1n}' = \frac{(m_1 - m_2)v_{1n} + 2m_2 v_{2n}}{m_1 + m_2}$$

$$v_{2n}' = \frac{(m_2 - m_1)v_{2n} + 2m_1 v_{1n}}{m_1 + m_2}$$

$$v_{1t}' = v_{1t}, \quad v_{2t}' = v_{2t}$$

### Coefficient of Restitution

For elastic collisions, the coefficient of restitution $e = 1$:

$$e = \frac{v_2' - v_1'}{v_1 - v_2} = 1$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from matplotlib.collections import PatchCollection

# Set random seed for reproducibility
np.random.seed(42)

## Particle Class Definition

We define a `Particle` class to encapsulate the physical properties and state of each particle in our simulation.

In [ ]:
class Particle:
    """A class representing a particle in 2D space."""
    
    def __init__(self, x, y, vx, vy, radius, mass):
        self.position = np.array([x, y], dtype=float)
        self.velocity = np.array([vx, vy], dtype=float)
        self.radius = radius
        self.mass = mass
        
        # Track trajectory for visualization
        self.trajectory = [self.position.copy()]
    
    def update(self, dt):
        """Update particle position based on velocity."""
        self.position += self.velocity * dt
        self.trajectory.append(self.position.copy())
    
    def kinetic_energy(self):
        """Calculate kinetic energy of the particle."""
        return 0.5 * self.mass * np.dot(self.velocity, self.velocity)
    
    def momentum(self):
        """Calculate momentum vector of the particle."""
        return self.mass * self.velocity

## Collision Detection and Response

We implement functions to detect collisions between particles and walls, and to resolve them using the elastic collision equations.

In [ ]:
def check_particle_collision(p1, p2):
    """Check if two particles are colliding."""
    distance = np.linalg.norm(p1.position - p2.position)
    return distance <= (p1.radius + p2.radius)

def resolve_particle_collision(p1, p2):
    """Resolve elastic collision between two particles."""
    # Vector from p1 to p2
    delta = p2.position - p1.position
    distance = np.linalg.norm(delta)
    
    if distance == 0:
        return  # Avoid division by zero
    
    # Normal and tangent unit vectors
    normal = delta / distance
    
    # Relative velocity
    rel_velocity = p1.velocity - p2.velocity
    
    # Relative velocity along collision normal
    vel_along_normal = np.dot(rel_velocity, normal)
    
    # Only resolve if particles are approaching
    if vel_along_normal > 0:
        return
    
    # Calculate impulse scalar (for elastic collision, e = 1)
    impulse = (2 * vel_along_normal) / (1/p1.mass + 1/p2.mass)
    
    # Apply impulse to velocities
    p1.velocity -= (impulse / p1.mass) * normal
    p2.velocity += (impulse / p2.mass) * normal
    
    # Separate overlapping particles
    overlap = (p1.radius + p2.radius) - distance
    if overlap > 0:
        separation = overlap / 2 * normal
        p1.position -= separation
        p2.position += separation

def resolve_wall_collision(particle, box_size):
    """Resolve collision with walls (elastic reflection)."""
    # Left wall
    if particle.position[0] - particle.radius < 0:
        particle.position[0] = particle.radius
        particle.velocity[0] = abs(particle.velocity[0])
    # Right wall
    if particle.position[0] + particle.radius > box_size:
        particle.position[0] = box_size - particle.radius
        particle.velocity[0] = -abs(particle.velocity[0])
    # Bottom wall
    if particle.position[1] - particle.radius < 0:
        particle.position[1] = particle.radius
        particle.velocity[1] = abs(particle.velocity[1])
    # Top wall
    if particle.position[1] + particle.radius > box_size:
        particle.position[1] = box_size - particle.radius
        particle.velocity[1] = -abs(particle.velocity[1])

## Simulation Setup

We initialize a system of particles with random positions, velocities, and varying masses/radii.

In [ ]:
# Simulation parameters
NUM_PARTICLES = 8
BOX_SIZE = 10.0
DT = 0.01  # Time step
NUM_STEPS = 1000

# Create particles with random properties
particles = []
for i in range(NUM_PARTICLES):
    # Random radius between 0.3 and 0.6
    radius = np.random.uniform(0.3, 0.6)
    # Mass proportional to area (2D simulation)
    mass = np.pi * radius**2
    
    # Random position (ensuring particle is inside box)
    x = np.random.uniform(radius, BOX_SIZE - radius)
    y = np.random.uniform(radius, BOX_SIZE - radius)
    
    # Random velocity
    vx = np.random.uniform(-3, 3)
    vy = np.random.uniform(-3, 3)
    
    particles.append(Particle(x, y, vx, vy, radius, mass))

print(f"Created {NUM_PARTICLES} particles")
print(f"Total initial kinetic energy: {sum(p.kinetic_energy() for p in particles):.4f}")
print(f"Total initial momentum: {sum(p.momentum() for p in particles)}")

## Run Simulation

We evolve the system forward in time, checking for collisions at each step and tracking conservation quantities.

In [ ]:
# Arrays to track conservation quantities
times = []
kinetic_energies = []
momenta_x = []
momenta_y = []

# Run simulation
for step in range(NUM_STEPS):
    # Record conservation quantities
    times.append(step * DT)
    total_ke = sum(p.kinetic_energy() for p in particles)
    total_momentum = sum(p.momentum() for p in particles)
    kinetic_energies.append(total_ke)
    momenta_x.append(total_momentum[0])
    momenta_y.append(total_momentum[1])
    
    # Update particle positions
    for particle in particles:
        particle.update(DT)
    
    # Check and resolve wall collisions
    for particle in particles:
        resolve_wall_collision(particle, BOX_SIZE)
    
    # Check and resolve particle-particle collisions
    for i in range(len(particles)):
        for j in range(i + 1, len(particles)):
            if check_particle_collision(particles[i], particles[j]):
                resolve_particle_collision(particles[i], particles[j])

print(f"Simulation complete: {NUM_STEPS} steps")
print(f"Final kinetic energy: {kinetic_energies[-1]:.4f}")
print(f"Energy conservation error: {abs(kinetic_energies[-1] - kinetic_energies[0]):.6f}")

## Visualization

We create a comprehensive visualization showing:
1. Particle trajectories and final positions
2. Conservation of kinetic energy over time
3. Conservation of momentum components over time

In [ ]:
# Create figure with subplots
fig = plt.figure(figsize=(14, 10))

# Subplot 1: Particle trajectories and final positions
ax1 = fig.add_subplot(2, 2, 1)
colors = plt.cm.tab10(np.linspace(0, 1, NUM_PARTICLES))

for i, particle in enumerate(particles):
    trajectory = np.array(particle.trajectory)
    ax1.plot(trajectory[:, 0], trajectory[:, 1], '-', 
             color=colors[i], alpha=0.5, linewidth=0.5)
    circle = Circle(particle.position, particle.radius, 
                   facecolor=colors[i], edgecolor='black', alpha=0.7)
    ax1.add_patch(circle)
    # Add velocity arrow
    ax1.arrow(particle.position[0], particle.position[1],
              particle.velocity[0]*0.3, particle.velocity[1]*0.3,
              head_width=0.15, head_length=0.1, fc=colors[i], ec='black')

ax1.set_xlim(-0.5, BOX_SIZE + 0.5)
ax1.set_ylim(-0.5, BOX_SIZE + 0.5)
ax1.set_xlabel('x position')
ax1.set_ylabel('y position')
ax1.set_title('Particle Trajectories and Final Positions')
ax1.set_aspect('equal')
ax1.grid(True, alpha=0.3)

# Draw box boundaries
ax1.plot([0, BOX_SIZE, BOX_SIZE, 0, 0], 
         [0, 0, BOX_SIZE, BOX_SIZE, 0], 'k-', linewidth=2)

# Subplot 2: Kinetic energy conservation
ax2 = fig.add_subplot(2, 2, 2)
ax2.plot(times, kinetic_energies, 'b-', linewidth=1.5)
ax2.axhline(y=kinetic_energies[0], color='r', linestyle='--', 
            label=f'Initial KE = {kinetic_energies[0]:.3f}')
ax2.set_xlabel('Time')
ax2.set_ylabel('Total Kinetic Energy')
ax2.set_title('Conservation of Kinetic Energy')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Subplot 3: Momentum conservation (x-component)
ax3 = fig.add_subplot(2, 2, 3)
ax3.plot(times, momenta_x, 'g-', linewidth=1.5, label='$p_x$')
ax3.plot(times, momenta_y, 'm-', linewidth=1.5, label='$p_y$')
ax3.axhline(y=momenta_x[0], color='g', linestyle='--', alpha=0.5)
ax3.axhline(y=momenta_y[0], color='m', linestyle='--', alpha=0.5)
ax3.set_xlabel('Time')
ax3.set_ylabel('Total Momentum')
ax3.set_title('Conservation of Momentum Components')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Subplot 4: Energy distribution among particles
ax4 = fig.add_subplot(2, 2, 4)
particle_ke = [p.kinetic_energy() for p in particles]
particle_labels = [f'P{i+1}' for i in range(NUM_PARTICLES)]
bars = ax4.bar(particle_labels, particle_ke, color=colors, edgecolor='black')
ax4.set_xlabel('Particle')
ax4.set_ylabel('Kinetic Energy')
ax4.set_title('Final Kinetic Energy Distribution')
ax4.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('plot.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nFigure saved to 'plot.png'")

## Analysis and Verification

Let us verify that our simulation correctly conserves the physical quantities expected in elastic collisions.

In [ ]:
# Compute conservation errors
initial_ke = kinetic_energies[0]
final_ke = kinetic_energies[-1]
ke_error = abs(final_ke - initial_ke) / initial_ke * 100

initial_px, initial_py = momenta_x[0], momenta_y[0]
final_px, final_py = momenta_x[-1], momenta_y[-1]

print("=" * 50)
print("CONSERVATION ANALYSIS")
print("=" * 50)
print(f"\nKinetic Energy:")
print(f"  Initial: {initial_ke:.6f}")
print(f"  Final:   {final_ke:.6f}")
print(f"  Relative Error: {ke_error:.4f}%")

print(f"\nMomentum (x-component):")
print(f"  Initial: {initial_px:.6f}")
print(f"  Final:   {final_px:.6f}")
print(f"  Absolute Error: {abs(final_px - initial_px):.6f}")

print(f"\nMomentum (y-component):")
print(f"  Initial: {initial_py:.6f}")
print(f"  Final:   {final_py:.6f}")
print(f"  Absolute Error: {abs(final_py - initial_py):.6f}")

print("\n" + "=" * 50)
if ke_error < 1.0:
    print("✓ Kinetic energy is well conserved (< 1% error)")
else:
    print("✗ Kinetic energy conservation needs improvement")
print("=" * 50)

## Conclusion

This simulation demonstrates the fundamental physics of elastic collisions in two dimensions. Key observations:

1. **Conservation of Energy**: The total kinetic energy of the system remains constant throughout the simulation, as expected for elastic collisions.

2. **Conservation of Momentum**: Both components of the total momentum vector are conserved, reflecting Newton's third law.

3. **Energy Transfer**: During collisions, kinetic energy is transferred between particles, but the total remains constant. The final energy distribution among particles depends on the collision sequence.

4. **Numerical Accuracy**: Small numerical errors may accumulate over many time steps, but the use of proper collision resolution algorithms keeps these errors minimal.

The simulation can be extended to study:
- Maxwell-Boltzmann velocity distributions in thermal equilibrium
- Brownian motion with massive vs. light particles
- Phase space dynamics and ergodicity